<a href="https://colab.research.google.com/github/Dimildizio/DS_course/blob/main/shine-on-you/HomeWork2_Regression%26Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Домашняя работа 2

## Регрессия

С 1-ого октября отменяется НДС на бриллианты, теперь они становятся новым инвестиционным инструментом. Давайте сделаем модель ценообразования для них.

Скачайте датасет diamonds.csv

В нем представлены характеристики бриллиантов и их цены с сайта jamesallen (B2C площадка) с 2022-07-01

**Описание полей**


* fluor - флуорисценция (свойство камня светиться в ультрафиолетовых лучах)
* symmetry - показатель симметричности
* platform - название платформы, где был размещен камень
* quality_group - составной показатель из cut polish symmetry
* size_group - каратно весовая группа
* big_size_group - каратно-весовая группа
* shape - форма
* color - цвет
* clarity - чистота
* cut - качество огранки (может быть только у круглых камней)
* polish - полировка
* id - номер камня
* date - дата
* price - цена
* carat - кол-во карат
* tablepercent - размер площадки по отношению ширине
* price_per_carat - цена за карат
* z - длина (diameter)
* x - ширина
* depth_perc - отношение высоты к ширине
* y - высота



In [ ]:
import pandas as pd
import numpy as np

In [ ]:
#from google.colab import drive
#drive.mount('/content/drive')
#path = 'drive/MyDrive/'

Mounted at /content/drive


In [176]:
df = pd.read_csv('https://raw.githubusercontent.com/Dimildizio/DS_course/main/shine-on-you/diamonds.csv', index_col=0)
df.shape

(222222, 17)

## EDA

In [ ]:
df.head(10)

,fluor,symmetry,platform,shape,color,clarity,cut,polish,id,date,price,carat,price_per_carat,z,x,depth_perc,y
135269,NaN,EX,jamesallen,PS,F,SI1,NaN,EX,13870838,202206,12850.0,1.55,8290.32,10.29,6.41,62.0,3.97420
48477,MED,EX,jamesallen,RD,H,VVS2,EX,EX,11725253,202207,7510.0,1.02,7362.75,6.41,6.45,62.5,4.03125
236786,NaN,EX,jamesallen,EM,H,IF,NaN,EX,14444347,202205,21220.0,2.01,10557.21,8.60,6.37,65.0,4.14050
235781,NaN,EX,jamesallen,RD,E,VS2,EX,EX,14438434,202207,8660.0,1.00,8660.00,6.39,6.44,61.4,3.95416
277744,NaN,VG,jamesallen,RD,F,VS1,VG,EX,14615276,202206,8480.0,1.00,8480.00,6.28,6.36,62.3,3.96228


In [177]:
#print('number of duplicates:',df.duplicated(keep=False).sum() - df.shape[0])
df1=df.copy()
df1=df1.drop(df1[df1.id == 0].index)
df1=df1.drop('platform', axis = 1)
df1['fluor'] = df1['fluor'].replace('NONE', np.nan)

In [178]:
for col in ['fluor', 'symmetry','shape', 'clarity', 'color', 'cut','polish']:
  df1[col] = df1[col].astype('category')
for col in ['id','date']:
  df1[col] = df1[col].astype('int32')
for col in ['price', 'carat','price_per_carat', 'z', 'x', 'y', 'depth_perc']:
  df1[col] = df1[col].astype('float32')

In [179]:
df1[df1.duplicated(keep=False)]

,fluor,symmetry,shape,color,clarity,cut,polish,id,date,price,carat,price_per_carat,z,x,depth_perc,y
451655,NaN,EX,RD,F,VVS2,EX,EX,15389091,202207,15120.0,1.25,12096.000000,6.88,6.90,61.900002,4.27110
320510,MED,EX,EM,G,VS2,NaN,EX,14767449,202207,101440.0,5.01,20247.500000,11.27,8.20,68.300003,5.60060
15140,FNT,VG,CUSH,K,I1,G,EX,5767007,202207,2320.0,1.03,2252.429932,6.61,6.04,57.099998,3.44884
61030,NaN,VG,EM,H,IF,NaN,EX,12350766,202207,20980.0,2.00,10490.000000,8.35,6.09,67.000000,4.08030
64160,NaN,EX,RD,G,VS2,G,EX,12463861,202207,5900.0,1.00,5900.000000,6.17,6.22,64.699997,4.02434
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
420457,NaN,VG,OV,E,SI2,NaN,EX,15174571,202207,3550.0,1.01,3514.850098,8.04,5.67,59.700001,3.38499
375122,NaN,VG,EM SQ,F,VVS2,NaN,EX,14974986,202207,12190.0,1.52,8019.740234,6.20,6.05,70.599998,4.27130
436661,FNT,EX,RD,E,VVS2,EX,EX,15282248,202207,44960.0,2.06,21825.240234,8.10,8.15,62.400002,5.08560
122116,NaN,VG,PR,E,VS1,EX,EX,13694872,202207,5230.0,1.02,5127.450195,5.52,5.46,73.900002,4.03494


In [180]:
df1.describe().apply(lambda x: round(x,2))

,id,date,price,carat,price_per_carat,z,x,depth_perc,y
count,222218.00,222218.00,222218.00,222218.00,222218.00,222218.00,222218.00,222218.00,222218.00
mean,13588555.40,202206.04,11715.41,1.34,7035.03,7.08,6.42,64.09,4.10
std,2547881.60,0.82,25403.57,0.72,5216.41,1.38,1.05,4.19,0.60
min,19423.00,202205.00,920.00,0.81,1029.70,4.52,3.87,46.00,2.47
25%,13598651.50,202205.00,4210.00,1.00,4148.51,6.26,5.69,61.80,3.81
50%,14400461.00,202206.00,6560.00,1.01,5851.49,6.53,6.38,63.00,4.00
75%,14837629.75,202207.00,11600.00,1.50,8609.27,7.66,6.82,66.00,4.32
max,15418506.00,202207.00,2380380.00,21.17,955000.00,21.88,18.05,78.00,10.74


In [183]:
for c in df1.columns:
  print(f'{c}: {df1[c].nunique()}')

fluor: 3
symmetry: 4
shape: 10
color: 10
clarity: 9
cut: 3
polish: 3
id: 162277
date: 3
price: 8436
carat: 631
price_per_carat: 52604
z: 1115
x: 903
depth_perc: 317
y: 37649


Необходимо предсказать `price_per_carat`

### Задание 1: Очистка

Не все камни успевают продаться за один месяц, поэтому в таблице есть повторы. Объедините данные по одному камню: подумайте, как лучше это сделать, какую цену брать.

**Попробуйте найти аномалии:** вдруг цена на некоторые камни сильно меняется (то есть продавец сам не знает, по какой цене их продавать). Также убедитесь, что остальные параметры камня не меняются.

In [231]:

# разделите выборку на две части

id_stat = df1.groupby(by='id')['id'].count().to_frame().rename({'id':'occurrence'},axis=1) #Посчитайте статистику по кол-ву камней

one = id_stat[id_stat['occurrence'].isin([1])].index       # те камни, которые встречались один раз
three = id_stat[id_stat['occurrence'].isin([2,3])].index   # те камни, которые встречались 2 или 3 раза

In [232]:
num_more_than_three = df1.shape[0] - (df1[df1['id'].isin(one)].shape[0] + df1[df1['id'].isin(three)].shape[0])
num_more_than_three

1167

In [234]:
df_one = df1[df1['id'].isin(one)]
df_three = df1[df1['id'].isin(three)]

df_three

,fluor,symmetry,shape,color,clarity,cut,polish,id,date,price,carat,price_per_carat,z,x,depth_perc,y
135269,NaN,EX,PS,F,SI1,NaN,EX,13870838,202206,12850.0,1.55,8290.320312,10.29,6.41,62.000000,3.97420
48477,MED,EX,RD,H,VVS2,EX,EX,11725253,202207,7510.0,1.02,7362.750000,6.41,6.45,62.500000,4.03125
236786,NaN,EX,EM,H,IF,NaN,EX,14444347,202205,21220.0,2.01,10557.209961,8.60,6.37,65.000000,4.14050
235781,NaN,EX,RD,E,VS2,EX,EX,14438434,202207,8660.0,1.00,8660.000000,6.39,6.44,61.400002,3.95416
277744,NaN,VG,RD,F,VS1,VG,EX,14615276,202206,8480.0,1.00,8480.000000,6.28,6.36,62.299999,3.96228
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
229133,NaN,EX,RD,I,SI2,EX,EX,14409793,202205,3330.0,0.90,3700.000000,6.13,6.18,62.000000,3.83160
268649,FNT,VG,RD,K,VS1,VG,EX,14581367,202205,3840.0,1.01,3801.979980,6.27,6.33,63.000000,3.98790
20595,NaN,EX,RD,D,VS2,EX,EX,7535786,202207,8700.0,0.97,8969.070312,6.36,6.32,62.200001,3.93104
282754,NaN,EX,RD,D,VS2,EX,EX,14631732,202206,6650.0,0.90,7388.890137,6.13,6.18,61.700001,3.81306


In [241]:
stat = df_three.groupby('id')['price', 'carat', 'price_per_carat','depth_perc', 'x','y','z'].agg(np.std)
stat.sort_values(by='carat', ascending = False)

<ipython-input-241-d2b98e5e63fd>:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  stat = df_three.groupby('id')['price', 'carat', 'price_per_carat','depth_perc', 'x','y','z'].agg(np.std)


,price,carat,price_per_carat,depth_perc,x,y,z
id,,,,,,,
14408689,106.066017,0.070711,582.182281,1.909189,0.056568,0.154128,0.035355
14772485,0.000000,0.056568,468.012158,0.070710,0.000000,0.004469,0.000000
10337722,178.978583,0.040415,61.510920,0.351189,0.092376,0.038917,0.057735
14040944,0.000000,0.035355,343.505431,0.070710,0.007071,0.000028,0.021213
14789652,0.000000,0.035355,388.533770,0.282844,0.049497,0.048182,0.042426
...,...,...,...,...,...,...,...
13700654,0.000000,0.000000,0.000000,0.070710,0.000000,0.003847,0.000000
13700657,113.137085,0.000000,113.137085,0.000000,0.000000,0.000000,0.000000
13700665,162.634560,0.000000,162.634560,0.212131,0.000000,0.013661,0.000000


In [250]:
stat[stat['price'] > 0].shape[0]

42766

In [ ]:

#df_one = df.query('...') 
#df_three = df.query('...') 


 # Посчитайте дстандартное отклонение по нескольким полям,
#  подумайте где оно должно равняться 0, а где меняться в каких-то разумных пределах

# Удалите аномальные наблюдения
df_three = df_three.query('...').sort_values(by=['id','date'])

df_three = df_three.groupby('...').agg('l...').reset_index() #тепреь в качестве цены возьмем последнее значение по времени

df = pd.concat([df_one, df_three]).reset_index(drop=True) # соединяем результаты

Цены на бриллианты достаточно сильно меняются, попробуйте вычислить коэффициент инфляции и привести цены к последнему месяцу. Стоит учесть, что цены на все бриллианты не изменяются синхронно, то есть изменение в определенных группах может быть разным.

*Определите эти группы и рассчитайте коэффициенты инфляции для каждой из них. Подправьте цены на эти коэффициенты и создайте новую переменную.*

In [ ]:
categories = ['...'] #определите по каким группам отпределять инфляцию (к примеру можно добавить каратную группу)

df_index = df.groupby(['date'] + categories)[['price_per_carat']]\
             .mean()\
             .reset_index() # индекс цен

date_max = df_index.date.max()


df_index = df_index.merge(df_index.query('date == @date_max')[categories + ['price_per_carat']]\
                          .rename(columns={'price_per_carat': 'price_per_carat_max'}), 
                          on=categories, how='outer') # сопоставляем группы с максимальной датой

df_index['inflation'] = df_index.['...'] / df_index.price_per_carat # вычисляем инфляцию

df_with_inf = df.merge(df_index[['date'] + categories + ['inflation']], 
                        on=['date'] + categories, how='left') # соединяем все в одной таблице

### Задание 2: Модель

Определите функцию потерь (MSE или MAE) и аргументируйте выбор.
Попробуйте сделать baseline.
Используйте LableEncoder для категориальных фичей и **постройте линейную модель**.

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LableEncoder
from sklearn.metrics import ...


X = df.drop(columns=["..."])
y = df["..."]

X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=13)

Теперь попробуйте OHE или TargetEncoder (сравните их).

Нормализуйте данные.

Поработайте с пропусками (обратите внимание на то, что у fluor возможен пропуск значения, а возможно отсутствие флуоресценции)

Покажите, насколько получилось улучшить результат.

In [ ]:
! pip install category_encoders

In [ ]:
from sklearn.preprocessing import OneHotEncoder
from category_encoders import TargetEncoder
from sklearn.preprocessing import StandardScaler

from typing import Tuple, List


def OHE(df: pd.DataFrame, columns: List[str]) -> Tuple[pd.DataFrame, List[str]]:
    """
    Вместо это функции можно использовать и функционал pandas
    """
    index = df.index
    one = OneHotEncoder(sparse=False, categories='auto')
    ohe = one.fit_transform(df[columns])
    col_names = one.get_feature_names(input_features = columns)
    df = df.drop(columns, axis=1)
    df = df.reset_index(drop=True)
    df = pd.concat([df, pd.DataFrame(ohe, columns=col_names)], axis = 1)
    df = df.set_index(index)
    return (df, col_names)

scaler = StandardScaler() # Помните, что на тесте делаем только transform

Сравните работу выбранной вами линейной модели с KNN

In [ ]:
from sklearn.neighbors import KNeighborsRegressor

...

## Классификация

Загрузите датасет bodyPerformance

Описание:

https://www.kaggle.com/datasets/kukuroo3/body-performance-data

In [ ]:
#from google.colab import drive
#drive.mount('/content/drive')

#path = 'drive/MyDrive/'

In [146]:
df = pd.read_csv('https://raw.githubusercontent.com/Dimildizio/DS_course/main/shine-on-you/bodyPerformance.csv', index_col=0) #укажите свой путь

In [166]:
df.head(20)

,gender,height_cm,weight_kg,body fat_%,diastolic,systolic,gripForce,sit and bend forward_cm,sit-ups counts,broad jump_cm,class
age,,,,,,,,,,,
27.0,M,172.3,75.24,21.3,80.0,130.0,54.9,18.4,60.0,217.0,C
25.0,M,165.0,55.80,15.7,77.0,126.0,36.4,16.3,53.0,229.0,A
31.0,M,179.6,78.00,20.1,92.0,152.0,44.8,12.0,49.0,181.0,C
32.0,M,174.5,71.10,18.4,76.0,147.0,41.4,15.2,53.0,219.0,B
28.0,M,173.8,67.70,17.1,70.0,127.0,43.5,27.1,45.0,217.0,B
36.0,F,165.4,55.40,22.0,64.0,119.0,23.8,21.0,27.0,153.0,B
42.0,F,164.5,63.70,32.2,72.0,135.0,22.7,0.8,18.0,146.0,D
33.0,M,174.9,77.20,36.9,84.0,137.0,45.9,12.3,42.0,234.0,B
54.0,M,166.8,67.50,27.6,85.0,165.0,40.4,18.6,34.0,148.0,C


In [171]:
df[df.duplicated()]

,gender,height_cm,weight_kg,body fat_%,diastolic,systolic,gripForce,sit and bend forward_cm,sit-ups counts,broad jump_cm,class
age,,,,,,,,,,,
27.0,F,157.0,49.1,30.7,70.0,86.0,27.7,19.7,51.0,167.0,A


### Задание 1: Определение гендера

Постройте модель, которая будет определять гендер.

Для этого определите две метрики качества, на которые вы будете смотреть. Аргументируйте свой выбор.

После чего преобразуйте категориальные переменные и постройте модель бинарной классификации на основе линейного классификатора и сравните с KNN.

P.S.: не забудте про нормализацию.

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
X = df.drop(columns=["..."])
y = df["..."]

X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=13)

scaler = StandardScaler()

### Задание 2: Определение класса

Теперь опробуем построить модель, которая будет предсказывать class физической формы.

Для этого определите метрики качества для задачи мультиклассификации (аргументируйте выбор).

Постройте модель мультиклассовой классификации на основе линейного классификатора и сравните с KNN.

In [ ]:
X = df.drop(columns=["..."]) # укажите новый таргет
y = df["..."]

X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=13)

In [ ]:
...